In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [25]:
memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [26]:
loader = TextLoader("./files/document.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=650,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using the following context:\n\n{context}\n\nand You refer to history:"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain =  {"context": retriver, "question": RunnablePassthrough()} | RunnablePassthrough.assign(history=load_memory) | prompt | llm

In [28]:
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

    return result.content

In [29]:
invoke_chain("Aaronson 은 유죄인가요?")

예, Aaronson은 자신들에게 적혀있는 죄를 저질렀다고 인정했습니다. 그러나 그의 기억은 조작되었고, 그의 유죄를 반증하는 사진은 실제로는 존재하지 않았습니다.

'예, Aaronson은 자신들에게 적혀있는 죄를 저질렀다고 인정했습니다. 그러나 그의 기억은 조작되었고, 그의 유죄를 반증하는 사진은 실제로는 존재하지 않았습니다.'

In [30]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

그가 쓴 메시지는 다음과 같습니다:
FREEDOM IS SLAVERY
TWO AND TWO MAKE FIVE
GOD IS POWER

그는 이 메시지를 적고 있을 때 자신의 생각을 수용하고 있음을 나타냅니다.

AIMessageChunk(content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER\n\n그는 이 메시지를 적고 있을 때 자신의 생각을 수용하고 있음을 나타냅니다.')

In [31]:
chain.invoke("Julia 는 누구인가요?")

Julia는 주인공과 함께 이야기하는 여성 캐릭터입니다. 그녀는 주인공과 함께 반정부 활동을 하며 사랑을 나누지만, 이야기 속에서 감옥에 갇히게 되고 서로를 배신하게 됩니다.

AIMessageChunk(content='Julia는 주인공과 함께 이야기하는 여성 캐릭터입니다. 그녀는 주인공과 함께 반정부 활동을 하며 사랑을 나누지만, 이야기 속에서 감옥에 갇히게 되고 서로를 배신하게 됩니다.')